In [1]:
import pandas as pd
import numpy as np

import dash
from dash import html, dcc, Input, Output, State
import joblib
from sklearn.cluster import KMeans
import networkx as nx
from sklearn.manifold import MDS
import plotly.graph_objects as go

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors

!!! Ce code n'est pas terminé. Il s'agit de la première tentative de combiner les données fournies et l'interface graphique crée sur les données personalisées. Le code est executable mais incomplet et nécessite un travail de gestion des informations du graphique crée à partir des données afin de le rendre fonctionnel !!!

In [2]:
#Implémentation du graphique dynamique par rapport aux données fournies

full_data = joblib.load("bertopic_results_data.joblib")         #Récupération et mise en forme des données
topic_info = full_data["topic_info"]

df=pd.read_csv("Proximity_Matrix.csv", sep=";") 
df_to_numpy = df.to_numpy()
dist_matrice = 1-df_to_numpy[:,1:]

In [3]:
#définition des paramètres principaux du problème

nom_topic = topic_info["Name"].explode()[:]
for k in range(50) : 
    nom_topic[k]=nom_topic[k][nom_topic[k].index('_')+1:].replace('_',' ')
taille_topic=topic_info["Count"].explode()

In [4]:
#Récupération des positions associés au graph des topics et de sa matrice de proximité 
#pour la méthode de Fruchterman-Reingold

Graph = nx.Graph()                        
for i in range(50):                   
    for j in range(i + 1, 50):        
        Graph.add_edge(i, j, dist=dist_matrice[i, j])
        
poids = {(u, v): 1 - d['dist'] for u, v, d in Graph.edges(data=True)} 
nx.set_edge_attributes(Graph, poids, name='poids')

pos = nx.spring_layout(Graph, weight='poids', seed=43)
coordonees = np.array([pos[i] for i in range(50)])

nombre_clusters = 5  
kmeans = KMeans(n_clusters=nombre_clusters, random_state=43)
labels = kmeans.fit_predict(coordonees)

Dans la suite du code, j'ai tenté d'adapter par étape le code fonctionnel pour les données personnalisées aux données sur les topics. 

In [5]:
color_map=['red', 'blue', 'green', 'yellow', 'purple']
node_color_map={}
for knom in range(50) : 
    node_color_map[nom_topic[k]]=color_map[labels[k]]

In [6]:
Graph = nx.Graph()    
for k in range(50) : 
    Graph.add_node(nom_topic[k])
    Graph.nodes[nom_topic[k]]['size'] = taille_topic[k]
#On rajoutera les liens après

for node in Graph.nodes():
    if 'size' not in Graph.nodes[node] : 
        Graph.nodes[node]['size']=min(taille_topic)

# for i in range(50):                   
#     for j in range(i + 1, 50):        
#         Graph.add_edge(i, j, dist=dist_matrice[i, j])  
        
poids = {(u, v): 1 - d['dist'] for u, v, d in Graph.edges(data=True)} 
nx.set_edge_attributes(Graph, poids, name='poids')

pos = nx.spring_layout(Graph, weight='poids', seed=43)

app = dash.Dash(__name__)
app.layout = html.Div([
    html.H1("Graph Topics"),
    dcc.Graph(id="graph")
])

@app.callback(
    Output("graph", "figure"),
    Input("graph", "hoverData"),
)

def update_graph(hoverData):    
    # Reconstruire le node_trace
    node_x, node_y, node_text, node_size, node_colors = [], [], [], [], []
    max_poids = max(taille_topic)
    for node in Graph.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        node_text.append(node)
        node_size.append(Graph.nodes[node]['size'] / max_poids * 50 + 10)
        node_colors.append(node_color_map.get(node, "gray"))
    
    node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode="markers+text",
    text=node_text,
    hoverinfo="text",
    customdata=node_text,
    marker=dict(
        color=node_colors,
        size=node_size,
        line_width=2
    ),
    textposition="top center"
    )

    fig = go.Figure(
            data=[node_trace],
            layout=go.Layout(
                showlegend=False,
                hovermode="closest",
                margin=dict(b=20, l=5, r=5, t=40),
                xaxis=dict(showgrid=False, zeroline=False),
                yaxis=dict(showgrid=False, zeroline=False),
            ),
        ) 
    # if relayoutData:
    #     if 'xaxis.range[0]' in relayoutData and 'xaxis.range[1]' in relayoutData:
    #         fig.update_xaxes(range=[
    #             relayoutData['xaxis.range[0]'],
    #             relayoutData['xaxis.range[1]']
    #         ])
    #     if 'yaxis.range[0]' in relayoutData and 'yaxis.range[1]' in relayoutData:
    #         fig.update_yaxes(range=[
    #             relayoutData['yaxis.range[0]'],
    #             relayoutData['yaxis.range[1]']
    #         ])

    if hoverData is None:
        return fig 

    # Récupérer le nœud survolé
    try:
        node_id = hoverData["points"][0]["customdata"]
    except (KeyError, IndexError, TypeError):
        # Fallback en cas de structure inattendue
        return dash.no_update

    # Arêtes liées à ce nœud
    #filtered_edges = [e for e in G.edges() if node_id in e]

    # Générer uniquement les segments des arêtes concernées
    # edge_x, edge_y = [], []
    # for edge in filtered_edges:
    #     x0, y0 = pos[edge[0]]
    #     x1, y1 = pos[edge[1]]
    #     edge_x += [x0, x1, None]
    #     edge_y += [y0, y1, None]

    # edge_trace = go.Scatter(
    #     x=edge_x,
    #     y=edge_y,
    #     line=dict(width=2, color="#888"),
    #     hoverinfo="none",
    #     mode="lines",
    # )
    
    # fig.add_trace(edge_trace)

    return fig 
    
if __name__ == "__main__":
    app.run(debug=True)
    
#Si l'erreur "Address 'http://127.0.0.1:8050' already in use." se présente, cela signifie simplement que deux versions 
# différentes de l'app tournent à la même adresse. Il faut simplement fermer les onglets en trop. A la limite, fermer vsc et le rouvrir.